In [1]:
from transformers import MBartForConditionalGeneration, MBartTokenizer, AdamW, MBartConfig

import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np

import torch.nn.functional as F
import torch

import math
import random
import re
import argparse

/home/yc429/.conda/envs/gpt2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import (
    MBartForConditionalGeneration, MBartTokenizer, MBartConfig)

# from transformers import MBartForConditionalGeneration, MBartTokenizer

from transformers import Seq2SeqTrainer
# from seq2seq_training_args import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainingArguments


In [3]:
tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-cc25",
                                           add_prefix_space=True, 
                                           src_lang="zh_CN",
                                           tgt_lang="zh_CN")

#special tokens
content_tokens = ["<c"+str(x)+">" for x in range(0,300)]
rhyme_tokens = ["<b"+str(x)+">" for x in range(0,4)]
new_tokens = ["<h>","<l>","<d>","<m>"]
new_tokens.append("<|sep|>")
new_tokens.append("<|num|>")
new_tokens.append("<|bdr|>")
new_tokens.append("<|kwd|>")
new_tokens += content_tokens
new_tokens += rhyme_tokens

tokenizer.add_tokens(list(new_tokens))


312

In [1]:
import regex as re

def contains_japanese_or_korean(text):
    # Regular expression pattern for Japanese and Korean characters
    pattern = re.compile(r'[\p{Script=Hiragana}\p{Script=Katakana}\p{Script=Hangul}]+')
    
    # Search for the pattern in the text
    match = pattern.search(text)
    
    # If a match is found, return True; otherwise, return False
    return match is not None

# Example usage:
text1 = "こんにちは 안녕하세요"
text2 = "Hello, World!"
text3 = "今天"

print(contains_japanese_or_korean(text1))  # True
print(contains_japanese_or_korean(text3))  # False

def remove_punctuation(string):
    # initializing punctuations string
    punc = '''!()-[]{};:'"\,./?@#$%^&*_~。，、！（）【】？'''
    # Removing punctuations in string
    # Using loop + punctuation string
    for ele in string:
        if ele in punc:
            string = string.replace(ele, "")
    return string

#training_data
data = list()

with open("./L_scasion_training_data/three_height_scansion_wb_tones.csv","r") as fin:
    lines = [line.strip() for line in fin.readlines()]
    for sent in lines[1:]:
        sent = sent.split("\t")
        keyword,wb, tones, lyric = sent[0],sent[1],sent[2],sent[3]
        if len(keyword) == 0:
            print("0 keywords")
        else:
            lyr = remove_punctuation(lyric)
            if 1 < len(lyr) < 20:
                if contains_japanese_or_korean(lyr):
                    pass
                else:
                    num =  ["<c" + str(x) + ">" for x in range(len(lyr))]
                    d = keyword + " <|kwd|> " + " ".join(num)  + " <|num|> "+ wb + " <bdr> " + tones + " <|sep|> " + lyr 
                    data.append(d)

                    

True
False


In [2]:
dataset = data
print(len(dataset),len(set(dataset)))

877868 877868


In [3]:
dataset[0]

'款人 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <c7> <c8> <c9> <c10> <c11> <c12> <c13> <c14> <c15> <|num|> <b1> <b1> <b0> <b1> <b1> <b1> <b0> <b1> <b1> <b1> <b0> <b1> <b0> <b1> <b1> <b1> <bdr> <l> <l> <h> <l> <m> <h> <h> <m> <l> <h> <l> <m> <m> <l> <l> <h> <|sep|> 我讲这款人这世人的爱比人什么卡多'

In [7]:
mbart_model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-cc25", 
                                                            cache_dir="/home/yc429/rds/hpc-work/jupyter/cache")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

mbart_model.resize_token_embeddings(len(tokenizer))
mbart_model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250339, 1024)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250339, 1024)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
      )

In [7]:
CN_dict = dict()
for d in dataset:
    line = d.split('<|sep|>')
    try:
        source, target = line[0], line[1].strip() #changed line[1:]
        CN_dict[source] = target
    except IndexError:
        print(line)   

In [8]:
data = []

for src, tgt in CN_dict.items():
    data.append(
          {
              "translation": {
                  "hi": str(src),
#                   "en": ' '.join(tgt)
                  "en":str(tgt)
              }
          }
      )

print(f'total size of data is {len(data)}')

total size of data is 814242


In [9]:
data[:2]

[{'translation': {'hi': '款人 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <c7> <c8> <c9> <c10> <c11> <c12> <c13> <c14> <c15> <|num|> <b1> <b1> <b0> <b1> <b1> <b1> <b0> <b1> <b1> <b1> <b0> <b1> <b0> <b1> <b1> <b1> <bdr> <l> <l> <h> <l> <m> <h> <h> <m> <l> <h> <l> <m> <m> <l> <l> <h> ',
   'en': '我讲这款人这世人的爱比人什么卡多'}},
 {'translation': {'hi': '格调 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <c7> <c8> <c9> <c10> <c11> <c12> <c13> <|num|> <b0> <b0> <b0> <b0> <b0> <b1> <b1> <b0> <b1> <b1> <b0> <b1> <b0> <b1> <bdr> <l> <h> <h> <l> <m> <l> <l> <m> <l> <h> <m> <h> <h> <h> ',
   'en': '百分之九十九点九九都格调太低'}}]

In [9]:
# Extract source and translated texts from the raw data
source_texts = [entry['translation']['hi'] for entry in data]
translated_texts = [entry['translation']['en'] for entry in data]


In [10]:
print(len(source_texts),len(translated_texts))

814242 814242


In [12]:
print(source_texts[:5],translated_texts[:5])

['款人 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <c7> <c8> <c9> <c10> <c11> <c12> <c13> <c14> <c15> <|num|> <b1> <b1> <b0> <b1> <b1> <b1> <b0> <b1> <b1> <b1> <b0> <b1> <b0> <b1> <b1> <b1> <bdr> <l> <l> <h> <l> <m> <h> <h> <m> <l> <h> <l> <m> <m> <l> <l> <h> ', '格调 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <c7> <c8> <c9> <c10> <c11> <c12> <c13> <|num|> <b0> <b0> <b0> <b0> <b0> <b1> <b1> <b0> <b1> <b1> <b0> <b1> <b0> <b1> <bdr> <l> <h> <h> <l> <m> <l> <l> <m> <l> <h> <m> <h> <h> <h> ', '伤口 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <|num|> <b0> <b1> <b0> <b1> <b1> <b0> <b1> <bdr> <m> <h> <h> <m> <l> <h> <l> ', '结果 <|kwd|> <c0> <c1> <c2> <c3> <c4> <|num|> <b0> <b1> <b1> <b0> <b1> <bdr> <m> <l> <h> <m> <h> ', '桃花羞 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <|num|> <b0> <b1> <b0> <b1> <b0> <b0> <b1> <bdr> <h> <h> <m> <h> <h> <h> <h> '] ['我讲这款人这世人的爱比人什么卡多', '百分之九十九点九九都格调太低', '不会愈合的伤口', '结果是遗憾', '只见桃花羞答答']


In [11]:
# splitting dataset into train, validation
import torch
from torch.utils.data import random_split

split = 0.1
train_dataset, eval_dataset = random_split(data, lengths=[int((1-split)*len(data)), len(data)-int((1-split)*len(data))])

# defining collator functioon for preparing batches on the fly ..

In [4]:
def data_collator(features:list):

  labels = [f["translation"]["en"] for f in features]
  inputs = [f["translation"]["hi"] for f in features]

  batch = tokenizer.prepare_seq2seq_batch(src_texts=inputs, src_lang="zh_CN", tgt_lang="zh_CN", tgt_texts=labels, max_length=128, max_target_length=128)

  for k in batch:
    batch[k] = torch.tensor(batch[k])

  return batch

# data_collator(data[:10])

In [5]:
import os

# Let's assume this is your output directory.
output_dir = "./mbart_3_height_wb_no_poem_epoch3"

# Find all subdirectories of output_dir.
subdirs = [os.path.join(output_dir, d) for d in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, d))]

# Sort them by last modification time.
subdirs = sorted(subdirs, key=os.path.getmtime)

# The last checkpoint is the most recent one.
last_checkpoint = subdirs[-1]

print(last_checkpoint)

mbart_model = MBartForConditionalGeneration.from_pretrained(last_checkpoint)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mbart_model.resize_token_embeddings(len(tokenizer))
mbart_model.to(device)
mbart_model.eval()

./mbart_3_height_wb_no_poem_epoch3/checkpoint-68000


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250339, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250339, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_

In [14]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./mbart_3_height_wb_no_poem_epoch3",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    num_train_epochs=3,
    save_steps=1000,
    save_total_limit=5,
)

# Create the Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=mbart_model,
    args=training_args,
    data_collator=data_collator,  # You can define a data collator if needed
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
    
)

# Start fine-tuning
trainer.train(last_checkpoint)

Loading model from ./mbart_3_height_wb_no_poem_epoch3/checkpoint-66000).
***** Running training *****
  Num examples = 732817
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 68703
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 2
  Continuing training from global step 66000
  Will skip the first 2 epochs then the first 20198 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.
Skipping the first batches:   0%|                             | 0/20198 [00:00<?, ?it/s]/home/yc429/.conda/envs/gpt2/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3415: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 o

Step,Training Loss,Validation Loss
66500,0.750200,0.709867
67000,0.750200,0.709360
67500,0.756900,0.708955
68000,0.743400,0.708723
68500,0.759200,0.708515


***** Running Evaluation *****
  Num examples = 81425
  Batch size = 32
***** Running Evaluation *****
  Num examples = 81425
  Batch size = 32
Saving model checkpoint to ./mbart_3_height_wb_no_poem_epoch3/checkpoint-67000
Configuration saved in ./mbart_3_height_wb_no_poem_epoch3/checkpoint-67000/config.json
Model weights saved in ./mbart_3_height_wb_no_poem_epoch3/checkpoint-67000/pytorch_model.bin
Deleting older checkpoint [mbart_3_height_wb_no_poem_epoch3/checkpoint-62000] due to args.save_total_limit
/home/yc429/.conda/envs/gpt2/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3415: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = toke

TrainOutput(global_step=68703, training_loss=0.029603517959692672, metrics={'train_runtime': 2916.7184, 'train_samples_per_second': 753.741, 'train_steps_per_second': 23.555, 'total_flos': 2.726523842875392e+17, 'train_loss': 0.029603517959692672, 'epoch': 3.0})

In [15]:
mbart_model.save_pretrained('./L_scansion_models/mbart_3_height_wb_no_poem_epoch3')

Configuration saved in ./L_scansion_models/mbart_3_height_wb_no_poem_epoch3/config.json
Model weights saved in ./L_scansion_models/mbart_3_height_wb_no_poem_epoch3/pytorch_model.bin


In [7]:
model_path = "./L_scansion_models/mbart_3_height_wb_no_poem_epoch3"
mbart_model = MBartForConditionalGeneration.from_pretrained(model_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mbart_model.resize_token_embeddings(len(tokenizer))
mbart_model.to(device)

mbart_model.eval()

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250339, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250339, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_

In [6]:
prompts = list()
with open("/home/yc429/Ch_lyrics_generation/12_new_melodies/12_prompts.txt","r") as fin:
    for line in fin.readlines()[1:]:
        line = line.strip().split("\t")
        melody_idx, keyword, length, wb, contour = line[0],line[1],int(line[2]),line[3],line[4]
        prompt = keyword + " <|kwd|> " + " ".join(["<c"+str(x)+">" for x in range(length)]) + " <|num|> " + wb + " <bdr> " + contour
        prompts.append(prompt)

In [7]:
def read_file_to_list(filepath):
    """
    Read a file and return each line as an element in a list.

    Parameters:
    - filepath (str): The path to the file to be read.

    Returns:
    - list of str: A list where each element is a line from the file.
    """
    with open(filepath, 'r') as file:
        # Read lines and strip newline characters
        lines = [line.strip().split("\t")[1] for line in file]

    return lines

# Example usage:
filepath = '/home/yc429/Ch_lyrics_generation/10_old_melodies/10_old_duration_wb_rule.txt'
lines_list = read_file_to_list(filepath)
print(lines_list)


['<b0> <b0> <b0> <b1> <b0> <b0> <b1>', '<b0> <b0> <b0> <b1> <b0> <b0> <b1>', '<b0> <b0> <b0> <b1> <b0> <b0> <b1> <b0>', '<b0> <b1> <b0> <b0> <b0>', '<b0> <b0> <b0> <b0> <b1> <b0> <b0> <b0> <b1> <b0>', '<b0> <b0> <b0> <b0> <b1> <b0> <b0> <b0> <b1> <b0>', '<b0> <b0> <b0> <b0> <b1> <b0> <b0> <b1> <b0>', '<b0> <b0> <b0> <b1> <b0> <b0> <b0> <b1> <b0>', '<b0> <b1> <b0> <b1> <b0> <b0> <b0> <b0> <b0> <b1>', '<b0> <b1> <b0> <b1> <b0> <b0> <b0> <b0> <b0> <b1>', '<b0> <b1> <b0> <b1> <b0> <b0> <b0> <b0> <b0> <b1>', '<b0> <b0> <b1> <b0> <b0> <b0> <b1> <b0>', '<b0> <b0> <b0> <b0> <b0> <b0> <b0> <b0>', '<b0> <b0> <b0> <b0> <b0> <b0>', '<b0> <b0> <b0> <b0> <b0> <b0> <b0> <b0>', '<b0> <b0> <b0> <b0> <b0> <b0>', '<b0> <b0> <b0> <b0> <b0> <b0> <b0> <b0> <b0>', '<b0> <b1> <b0> <b1> <b0> <b1> <b0> <b0> <b1> <b0>', '<b0> <b0> <b0> <b0> <b0> <b0> <b0> <b0> <b0>', '<b0> <b0> <b0> <b0> <b0> <b0>', '<b0> <b0> <b0> <b0> <b1> <b0> <b0> <b1> <b0>', '<b0> <b0> <b0> <b0> <b1> <b0> <b0> <b1> <b0>', '<b0> <b0> <b0> <b

In [8]:
prompts = list()
with open("/home/yc429/Ch_lyrics_generation/10_old_melodies/3_height_similarity_10_melody_prompts.txt","r") as fin:
    for line in fin.readlines()[1:]:
        line = line.strip().split("\t")
        prompt = line[1]
        prompts.append(prompt)
        # melody_idx, keyword, length, wb, contour = line[0],line[1],int(line[2]),line[3],line[4]
        # prompt = keyword + " <|kwd|> " + " ".join(["<c"+str(x)+">" for x in range(length)]) + " <|num|> " + wb + " <bdr> " + contour
        # prompts.append(prompt)

In [9]:
prompts_wb = list()
for i in range(len(prompts)):
    p = prompts[i].split("<|num|>")
    tmp = p[0] + "<|num|> " + lines_list[i] + " <bdr>" + p[1]
    prompts_wb.append(tmp)
    

In [10]:
three_height_generation = list()

for i in range(len(prompts_wb)):
    word_list = [prompts_wb[i]]
    emb_list = []
    for item in word_list:
        emb_list.append({'translation': {'hi': item,'en': ''}})
    A = data_collator(emb_list)
    print(A)
    print(A['input_ids'])

    fea_output = {}
    for i in range(A['input_ids'].shape[0]):
        B = torch.unsqueeze(A['input_ids'][i],0)
        output = mbart_model.generate(B.to(device='cuda'), max_length=500,num_beams=5, num_return_sequences=5)
        fea_output[word_list[i]] = tokenizer.batch_decode(output, skip_special_tokens=True,temperature=0.5)
    #     print(word_list[i],tokenizer.batch_decode(output, skip_special_tokens=True)[0])
        print(fea_output)
    three_height_generation.append(fea_output)
    

/home/yc429/.conda/envs/gpt2/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3415: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


{'input_ids': tensor([[     6,   3759,   5148, 250034, 250035, 250036, 250037, 250038, 250039,
         250040, 250041, 250032, 250335, 250335, 250335, 250336, 250335, 250335,
         250336,   4426,  34145,     42,   2740, 250030, 250028, 250030, 250027,
         250027, 250027, 250027, 250031,      2, 250025]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[     2, 250025]])}
tensor([[     6,   3759,   5148, 250034, 250035, 250036, 250037, 250038, 250039,
         250040, 250041, 250032, 250335, 250335, 250335, 250336, 250335, 250335,
         250336,   4426,  34145,     42,   2740, 250030, 250028, 250030, 250027,
         250027, 250027, 250027, 250031,      2, 250025]])
{'无情 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <|num|> <b0> <b0> <b0> <b1> <b0> <b0> <b1> <bdr> <m> <l> <m> <h> <h> <h> <h> <|sep|>': ['无语无泪爱上毒', '无语无声爱上毒', '无语无声爱上你', '无语无泪爱不义', '无语无泪心花瓣']}
{'input_ids':

In [11]:
three_height_generation

[{'无情 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <|num|> <b0> <b0> <b0> <b1> <b0> <b0> <b1> <bdr> <m> <l> <m> <h> <h> <h> <h> <|sep|>': ['无语无泪爱上毒',
   '无语无声爱上毒',
   '无语无声爱上你',
   '无语无泪爱不义',
   '无语无泪心花瓣']},
 {'泪水 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <c7> <|num|> <b0> <b0> <b0> <b1> <b0> <b0> <b1> <bdr> <m> <l> <m> <h> <h> <m> <h> <l> <|sep|>': ['流血流汗流泪水滴',
   '难舍难分流泪水淌',
   '无悔无怨流泪水滴',
   '无法平息流泪水',
   '好久不见流泪水']},
 {'眼神 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <|num|> <b0> <b0> <b0> <b1> <b0> <b0> <b1> <b0> <bdr> <m> <h> <h> <m> <l> <m> <|sep|>': ['含苞欲放迷了眼神',
   '不屑一顾谁的眼神',
   '含苞欲放红眼神',
   '不屑一顾无眼神',
   '含苞欲放红眼神迷']},
 {'风雨 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <c7> <c8> <c9> <|num|> <b0> <b1> <b0> <b0> <b0> <bdr> <h> <m> <m> <m> <m> <l> <l> <l> <m> <l> <|sep|>': ['风雨来来往往我无所畏',
   '任凭寒来暑往我们迎着风',
   '任凭来来回回的有雨有风',
   '任凭来来回回的有无有',
   '任凭来来回回的有雨淋着风']},
 {'结果 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <c7> <c8> <c9> <|num|> <b0> <b0> <b0> <b0> <b1> <b0> <b0> <b0> <b1> <b0> <bdr> <

In [ ]:
three_height_generation = list()

for i in range(len(prompts)):
    word_list = [prompts[i]]
    emb_list = []
    for item in word_list:
        emb_list.append({'translation': {'hi': item,'en': ''}})
    A = data_collator(emb_list)
    print(A)
    print(A['input_ids'])

    fea_output = {}
    for i in range(A['input_ids'].shape[0]):
        B = torch.unsqueeze(A['input_ids'][i],0)
        output = mbart_model.generate(B.to(device='cuda'), max_length=500,num_beams=5, num_return_sequences=5)
        fea_output[word_list[i]] = tokenizer.batch_decode(output, skip_special_tokens=True,temperature=0.5)
    #     print(word_list[i],tokenizer.batch_decode(output, skip_special_tokens=True)[0])
        print(fea_output)
    three_height_generation.append(fea_output)
    

In [ ]:
midi_idx_keywords = dict()
with open("/home/yc429/Ch_lyrics_generation/12_new_melodies/keywords_random_12_group.txt","r") as fin:
    idx = 1
    for line in fin.readlines()[1:]:
        line = line.strip()
        keywords = line.split("\t")[0].split()
        midi_idx_keywords[idx] = ",".join(keywords)
        idx += 1

def swap_keys_values(dictionary):
    swapped_dict = {value: key for key, value in dictionary.items()}
    return swapped_dict

keywords2midi = swap_keys_values(midi_idx_keywords)
print(keywords2midi)



for x in three_height_generation:
    keyword = list(x.keys())[0].split(" <|kwd|> ")[0]
    for key in keywords2midi.keys():
        if keyword in key:
            idx = keywords2midi[key]
    print(idx, keyword,list(x.values())[0][0])

In [1]:
for x in three_height_generation:
    keyword = list(x.keys())[0].split(" <kwd> ")[0]
    for key in keywords2midi.keys():
        if keyword in key:
            idx = keywords2midi[key]
    print(idx, keyword,list(x.values())[0][3])

NameError: name 'three_height_generation' is not defined

In [ ]:
for x in three_height_generation:
    keyword = list(x.keys())[0].split(" <kwd> ")[0]
    for key in keywords2midi.keys():
        if keyword in key:
            idx = keywords2midi[key]
    print(idx, keyword,list(x.values())[0][3])

In [32]:
mbart_model.eval()

word_list = ["笛子 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <c7> <|num|> <b0> <b1> <b0> <b1> <b0> <b1> <b0> <b1> <bdr> <h> <m> <h> <h> <m> <m> <h> <h> "]
emb_list = []
for item in word_list:
    emb_list.append({'translation': {'hi': item,'en': ''}})
A = data_collator(emb_list)
print(A)
print(A['input_ids'])

fea_output = {}
for i in range(A['input_ids'].shape[0]):
    B = torch.unsqueeze(A['input_ids'][i],0)
    output = mbart_model.generate(B.to(device='cuda'), max_length=500,num_beams=5, num_return_sequences=5)
    fea_output[word_list[i]] = tokenizer.batch_decode(output, skip_special_tokens=True,temperature=0.5)
#     print(word_list[i],tokenizer.batch_decode(output, skip_special_tokens=True)[0])
    print(fea_output)
    

{'input_ids': tensor([[     6, 229051,   1344, 250034, 250035, 250036, 250037, 250038, 250039,
         250040, 250041, 250042, 250032, 250335, 250336, 250335, 250336, 250335,
         250336, 250335, 250336,   4426,  34145,     42,   2740, 250027, 250030,
         250027, 250027, 250030, 250030, 250027, 250027,      2, 250025]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[     2, 250025]])}
tensor([[     6, 229051,   1344, 250034, 250035, 250036, 250037, 250038, 250039,
         250040, 250041, 250042, 250032, 250335, 250336, 250335, 250336, 250335,
         250336, 250335, 250336,   4426,  34145,     42,   2740, 250027, 250030,
         250027, 250027, 250030, 250030, 250027, 250027,      2, 250025]])
{'笛子 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <c7> <|num|> <b0> <b1> <b0> <b1> <b0> <b1> <b0> <b1> <bdr> <h> <m> <h> <h> <m> <m> <h> <h> ': ['太阳落山笛儿悠悠', '这时一只笛子吹过', 

In [6]:
import os

# Let's assume this is your output directory.
output_dir = "./mbart_3_height_epoch3"

# Find all subdirectories of output_dir.
subdirs = [os.path.join(output_dir, d) for d in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, d))]

# Sort them by last modification time.
subdirs = sorted(subdirs, key=os.path.getmtime)

# The last checkpoint is the most recent one.
last_checkpoint = subdirs[-1]

print(last_checkpoint)

mbart_model = MBartForConditionalGeneration.from_pretrained(last_checkpoint)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mbart_model.resize_token_embeddings(len(tokenizer))
mbart_model.to(device)
mbart_model.eval()

FileNotFoundError: [Errno 2] No such file or directory: './mbart_3_height_epoch3'

In [16]:
import os

# Let's assume this is your output directory.
output_dir = "./mbart_4_height_epoch3"

# Find all subdirectories of output_dir.
subdirs = [os.path.join(output_dir, d) for d in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, d))]

# Sort them by last modification time.
subdirs = sorted(subdirs, key=os.path.getmtime)

# The last checkpoint is the most recent one.
last_checkpoint = subdirs[-1]

print(last_checkpoint)

mbart_model = MBartForConditionalGeneration.from_pretrained(last_checkpoint)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mbart_model.resize_token_embeddings(len(tokenizer))
mbart_model.to(device)
mbart_model.eval()

./mbart_4_height_epoch3/checkpoint-57000


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250339, 1024)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250339, 1024)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
      )

In [17]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./mbart_3_height_1017_epoch3",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    num_train_epochs=3,
    save_steps=1000,
    save_total_limit=5,
)

# Create the Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=mbart_model,
    args=training_args,
    data_collator=data_collator,  # You can define a data collator if needed
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
    
)

# Start fine-tuning
trainer.train(last_checkpoint)

Loading model from ./mbart_4_height_epoch3/checkpoint-57000).


RuntimeError: Error(s) in loading state_dict for MBartForConditionalGeneration:
	size mismatch for final_logits_bias: copying a param with shape torch.Size([1, 250349]) from checkpoint, the shape in current model is torch.Size([1, 250339]).
	size mismatch for model.shared.weight: copying a param with shape torch.Size([250349, 1024]) from checkpoint, the shape in current model is torch.Size([250339, 1024]).
	size mismatch for model.encoder.embed_tokens.weight: copying a param with shape torch.Size([250349, 1024]) from checkpoint, the shape in current model is torch.Size([250339, 1024]).
	size mismatch for model.decoder.embed_tokens.weight: copying a param with shape torch.Size([250349, 1024]) from checkpoint, the shape in current model is torch.Size([250339, 1024]).
	size mismatch for lm_head.weight: copying a param with shape torch.Size([250349, 1024]) from checkpoint, the shape in current model is torch.Size([250339, 1024]).

***** Running Evaluation *****
  Num examples = 65274
  Batch size = 32
***** Running Evaluation *****
  Num examples = 65274
  Batch size = 32
Saving model checkpoint to ./mbart_3_height_1017_epoch3/checkpoint-36000
Configuration saved in ./mbart_3_height_1017_epoch3/checkpoint-36000/config.json
Model weights saved in ./mbart_3_height_1017_epoch3/checkpoint-36000/pytorch_model.bin
Deleting older checkpoint [mbart_3_height_1017_epoch3/checkpoint-31000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 65274
  Batch size = 32
***** Running Evaluation *****
  Num examples = 65274
  Batch size = 32
Saving model checkpoint to ./mbart_3_height_1017_epoch3/checkpoint-37000
Configuration saved in ./mbart_3_height_1017_epoch3/checkpoint-37000/config.json
Model weights saved in ./mbart_3_height_1017_epoch3/checkpoint-37000/pytorch_model.bin
Deleting older checkpoint [mbart_3_height_1017_epoch3/checkpoint-32000] due to args.save_total_limit
***** Running Evaluation ***

In [ ]:
mbart_model.save_pretrained('./3_height_epoch_3_1017')

In [31]:
word_list = ["夏天 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <|num|> <l> <h> <m> <h> <m> <h> <l> "]
emb_list = []
for item in word_list:
    emb_list.append({'translation': {'hi': item,'en': ''}})
A = data_collator(emb_list)
print(A)
print(A['input_ids'])

fea_output = {}
for i in range(A['input_ids'].shape[0]):
    B = torch.unsqueeze(A['input_ids'][i],0)
    output = mbart_model.generate(B.to(device='cuda'), max_length=500,num_beams=5, num_return_sequences=5)
    fea_output[word_list[i]] = tokenizer.batch_decode(output, skip_special_tokens=True,temperature=0.5)
#     print(word_list[i],tokenizer.batch_decode(output, skip_special_tokens=True)[0])
    print(fea_output)

{'input_ids': tensor([[     6,  68132, 250034, 250035, 250036, 250037, 250038, 250039, 250040,
         250041, 250032, 250028, 250027, 250030, 250027, 250030, 250027, 250028,
              2, 250025]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[     2, 250025]])}
tensor([[     6,  68132, 250034, 250035, 250036, 250037, 250038, 250039, 250040,
         250041, 250032, 250028, 250027, 250030, 250027, 250030, 250027, 250028,
              2, 250025]])
{'夏天 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <|num|> <l> <h> <m> <h> <m> <h> <l> ': ['把爱留在夏天里', '把爱埋在夏天里', '把爱埋进夏天里', '想念随季节变冷', '想念如夏天的雨']}


In [6]:
mbart_model.eval()
prompts = list()
with open("./M-scansion_inference_10_melody_prompts/4_height_similarity_10_melody_prompts.txt","r") as fin:
    for line in fin.readlines():
        prompts.append(" ".join(line.strip().split("\t")[-1].split()[:-1]))


In [7]:
midi2keywords = {1: '黑夜,无情,泪水,眼神',
 2: '风雨,结果,心情,朋友',
 3: '思念,日子,世界,时光',
 4: '女人,笑容,爱情,声音',
 5: '感觉,流泪,时候,眼睛',
 6: '梦想,世间,地方,流浪',
 7: '天空,心痛,时间,人生',
 8: '伤心,爱人,感情,记忆',
 9: '习惯,滋味,情人,勇气',
 10: '故事,青春,美丽,命运'}

def swap_keys_values(dictionary):
    swapped_dict = {value: key for key, value in dictionary.items()}
    return swapped_dict

keywords2midi = swap_keys_values(midi2keywords)
print(keywords2midi)

def output_lyrics(prompt,model):
    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
    generated = generated.to(device)
    sample_outputs = model.generate(
                            generated, 
                            #bos_token_id=random.randint(1,30000),
                            do_sample=True,   
                            top_k=10, 
                            max_length = 300,
                            top_p=0.95, 
                            num_return_sequences=1
                            )

    res = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
    return res

{'黑夜,无情,泪水,眼神': 1, '风雨,结果,心情,朋友': 2, '思念,日子,世界,时光': 3, '女人,笑容,爱情,声音': 4, '感觉,流泪,时候,眼睛': 5, '梦想,世间,地方,流浪': 6, '天空,心痛,时间,人生': 7, '伤心,爱人,感情,记忆': 8, '习惯,滋味,情人,勇气': 9, '故事,青春,美丽,命运': 10}


In [10]:
three_height_generation = list()

for i in range(len(prompts)):
    word_list = [prompts[i]]
    emb_list = []
    for item in word_list:
        emb_list.append({'translation': {'hi': item,'en': ''}})
    A = data_collator(emb_list)
    print(A)
    print(A['input_ids'])

    fea_output = {}
    for i in range(A['input_ids'].shape[0]):
        B = torch.unsqueeze(A['input_ids'][i],0)
        output = mbart_model.generate(B.to(device='cuda'), max_length=500,num_beams=5, num_return_sequences=5)
        fea_output[word_list[i]] = tokenizer.batch_decode(output, skip_special_tokens=True,temperature=0.5)
    #     print(word_list[i],tokenizer.batch_decode(output, skip_special_tokens=True)[0])
        print(fea_output)
    three_height_generation.append(fea_output)

/home/yc429/.conda/envs/gpt2/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3415: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


{'input_ids': tensor([[     6,   7320,   8097, 250034, 250035, 250036, 250037, 250038, 250039,
         250040, 250041, 250032, 250028, 250029, 250028, 250027, 250030, 250028,
         250030,      2, 250025]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[     2, 250025]])}
tensor([[     6,   7320,   8097, 250034, 250035, 250036, 250037, 250038, 250039,
         250040, 250041, 250032, 250028, 250029, 250028, 250027, 250030, 250028,
         250030,      2, 250025]])
{'黑夜 <|kwd|> <c0> <c1> <c2> <c3> <c4> <c5> <c6> <|num|> <l> <d> <l> <h> <m> <l> <m>': ['守着你黑夜很长', '守着你黑夜两茫', '有了你黑夜远离', '守着你黑夜旅行', '守着你黑夜两头']}
{'input_ids': tensor([[     6,   3759,   5148, 250034, 250035, 250036, 250037, 250038, 250039,
         250040, 250041, 250032, 250028, 250029, 250028, 250027, 250030, 250027,
         250027,      2, 250025]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': te

In [11]:
for x in three_height_generation:
    keyword = list(x.keys())[0].split("<|kwd|>")[0][:-1]
    sent = list(x.values())[0][2]
    for k in keywords2midi.keys():
        if keyword in k:
            idx = keywords2midi[k]
    print(keyword,sent,idx)

黑夜 有了你黑夜远离 1
无情 你的每句无情句 1
泪水 你的眼中泪水流着 1
眼神 眼神迷了眼 1
风雨 风雨来临时啊啊啊啊 2
结果 为何还执着着了的结果里 2
心情 别让心情随着时间流 2
朋友 来吧朋友们相聚相依偎 2
思念 思想起从前你离我远吗 3
日子 未来的男人要活在日子 3
世界 世界很神奇我能给你的 3
时光 女时光已模糊了你 3
女人 女人从来不怕孤独 4
笑容 用笑容说明了 4
爱情 我明白这就是爱情 4
声音 你啊你没声音 4
感觉 感觉是那么的暖洋洋 5
流泪 女流泪为了相恋到天明 5
时候 有时候真的了不起劲 5
眼睛 我不要闭眼睛 5
梦想 寻找着属于执着追逐 6
世间 才懂得远离繁华世间 6
地方 同一个儿同一个啊 6
流浪 我和你流浪流浪 6
天空 仰望天空翱翔和放纵 7
心痛 把心痛来折磨 7
时间 我知道爱没随时间去 7
人生 笑看人生的苦 7
伤心 掩饰着人们的惶恐 8
爱人 想念着咱们的爱人 8
感情 考验着人们的感情 8
记忆 童年里的童年你的童年 8
习惯 幸福的习惯你改改吧 9
滋味 缠绵的滋味谁能明白 9
情人 做个情人其实还没什么 9
勇气 把执着当作是那份勇气无穷 9
故事 你的故事从来没别人的 10
青春 有了你生活也无悔青春 10
美丽 你的美丽还在不在前 10
命运 是谁是命运的船 10


In [64]:
# defining training related arguments
args = Seq2SeqTrainingArguments(output_dir="4_height_num_wb_epoch",overwrite_output_dir=True,
                        do_train=True,
                        do_eval=True,
                        save_steps=100000,
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=32,
                        per_device_eval_batch_size=32,
                        learning_rate=5e-5,
                        num_train_epochs=3,
                        logging_dir="logs")

# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=mbart_model, 
                args=args, 
                data_collator=data_collator, 
                train_dataset=train_dataset, 
                eval_dataset=eval_dataset)

In [3]:
print(transformers.__version__)

4.13.0


In [15]:
print(transformers.__version__)

4.13.0


# Inference

In [10]:
midi2keywords = {1: '黑夜,无情,泪水,眼神',
 2: '风雨,结果,心情,朋友',
 3: '思念,日子,世界,时光',
 4: '女人,笑容,爱情,声音',
 5: '感觉,流泪,时候,眼睛',
 6: '梦想,世间,地方,流浪',
 7: '天空,心痛,时间,人生',
 8: '伤心,爱人,感情,记忆',
 9: '习惯,滋味,情人,勇气',
 10: '故事,青春,美丽,命运'}

def swap_keys_values(dictionary):
    swapped_dict = {value: key for key, value in dictionary.items()}
    return swapped_dict

keywords2midi = swap_keys_values(midi2keywords)
print(keywords2midi)

{'黑夜,无情,泪水,眼神': 1, '风雨,结果,心情,朋友': 2, '思念,日子,世界,时光': 3, '女人,笑容,爱情,声音': 4, '感觉,流泪,时候,眼睛': 5, '梦想,世间,地方,流浪': 6, '天空,心痛,时间,人生': 7, '伤心,爱人,感情,记忆': 8, '习惯,滋味,情人,勇气': 9, '故事,青春,美丽,命运': 10}


In [ ]:
def output_lyrics(prompt,model):
    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
    generated = generated.to(device)
    sample_outputs = model.generate(
                            generated, 
                            #bos_token_id=random.randint(1,30000),
                            do_sample=True,   
                            top_k=10, 
                            max_length = 300,
                            top_p=0.95, 
                            num_return_sequences=1
                            )

    res = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
    return res